## **1. Búsqueda y Scrapping de tweets**

------------------

Utilizamos ```Selenium``` con un navegador Chrome configurado para automatizar la búsqueda de tweets en Twitter (X) relacionados con una lista de palabras clave específicas sobre el trap argentino. El script navega a la página de búsqueda de Twitter para cada palabra clave, realiza desplazamientos para cargar más tweets y extrae información relevante de cada tweet, como el usuario, contenido, fecha, enlaces, menciones, retweets y likes. Luego, los datos recolectados se almacenan en un DataFrame de pandas y se guardan en un archivo CSV para su análisis posterior. 

---------------

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys  # Importar Keys
import undetected_chromedriver as uc
import re

# Configuración de opciones para Chrome
options = uc.ChromeOptions()
options.add_argument(r"--user-data-dir=C:\Users\adria\AppData\Local\Google\Chrome\User Data")  # Ruta al perfil de usuario de Chrome
options.add_argument(r"--profile-directory=Default")  # Selección del perfil de Chrome

# Iniciar navegador sin headless para ver si todo funciona bien
driver = uc.Chrome(options=options, version_main=135)  # Inicia un navegador Chrome con las opciones configuradas

# Lista de palabras clave relacionadas con estafas
keywords_list = [
    # Palabras clave relacionadas con el trap argentino, artistas y eventos
    "trap argentino",
    "música urbana argentina",
    "freestyle argentino",
    "escena trap argentina",
    "cultura trap argentina",
    'Omar Varela polémica',
    "Duki",
    "Bizarrap",
    "Trueno",
    "KHEA",
    "YSY A",
    "Cazzu",
    "Nicki Nicole",
    "Lit Killah",
    "Tiago PZK",
    "Neo Pistea",
    "Ecko",
    "L-Gante",
    "Bhavi",
    "CRO",
    'Ramma', 'Sixto Yegros', 'ACRU', 'Lucho SSJ',
    "Quinto Escalón",
    "BZRP Music Sessions",
    "Modo Diablo",
    "FMS Argentina",
    'Barderos',
    'Homer el Mero Mero Ysy A',
    "Cosquín Rock",
    "Movistar Arena",
    "La Renga y el trap",
    'polémica trap argentino',
    "Duki nuevo tema",
    "Bizarrap session",
    "YSY A videoclip",
    "freestyle Quinto Escalón",
    "Bizarrap ft",
    "escena trap Argentina"
]

# Lista para almacenar los tweets
tweets_data = []

# Número de desplazamientos para cargar más tweets
scrolls = 5  # Número de veces que se hará scroll hacia abajo en la página

# Bucle para hacer una búsqueda para cada palabra clave
for keyword in keywords_list:
    # URL de búsqueda en Twitter (X)
    driver.get(f"https://twitter.com/search?q={keyword}&src=typed_query&f=live")  # Abre la página de búsqueda en Twitter
    time.sleep(5)  # Espera a que cargue la página

    for _ in range(scrolls):
        # Extraer tweets
        tweet_elements = driver.find_elements(By.XPATH, '//article[@data-testid="tweet"]')  # Encuentra los tweets en la página

        for tweet in tweet_elements:
            try:
                # Usuario
                user = tweet.find_element(By.XPATH, './/span[contains(text(), "@")]').text  # Extrae el nombre de usuario
                # Contenido del tweet
                content = tweet.find_element(By.XPATH, './/div[@lang]').text  # Extrae el contenido del tweet
                # Fecha y hora
                date_time = tweet.find_element(By.XPATH, './/time').get_attribute('datetime')  # Extrae la fecha y hora del tweet
                # Enlaces en el tweet
                links = [link.get_attribute('href') for link in tweet.find_elements(By.XPATH, './/a')]  # Extrae los enlaces del tweet
                # Respuestas y menciones
                mentions = re.findall(r'@\w+', content)  # Busca menciones en el contenido del tweet
                # Retweets y likes (visibles en la parte inferior del tweet)
                retweets = tweet.find_element(By.XPATH, './/div[@data-testid="retweet"]').text if tweet.find_elements(By.XPATH, './/div[@data-testid="retweet"]') else '0'  # Extrae el número de retweets
                likes = tweet.find_element(By.XPATH, './/div[@data-testid="like"]').text if tweet.find_elements(By.XPATH, './/div[@data-testid="like"]') else '0'  # Extrae el número de likes
                
                # Guardar los datos extraídos
                tweets_data.append({
                    "Palabra Clave": keyword,  # Añadimos la palabra clave
                    "Usuario": user,
                    "Contenido": content,
                    "Fecha y Hora": date_time,
                    "Enlaces": links,
                    "Respuestas/Menciones": mentions,
                    "Retweets": retweets,
                    "Likes": likes
                })
            except Exception as e:
                print(f"Error al procesar un tweet: {e}")  # Manejo de errores en caso de que falle la extracción
                continue

        # Hacer scroll
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)  # Simula la tecla END para hacer scroll hacia abajo
        time.sleep(5)  # Espera para cargar más tweets

# Guardar los datos en un DataFrame
df = pd.DataFrame(tweets_data)  # Convierte los datos recolectados en un DataFrame de pandas

# Guardar en CSV para análisis posterior
df.to_csv("tweets_trap_argentino.csv", index=False)  # Guarda los datos en un archivo CSV

# Imprimir el número de tweets recolectados
print(f"Se han recolectado {len(df)} tweets.")  # Muestra cuántos tweets se recolectaron

# Cerrar el navegador
driver.quit()  # Cierra el navegador


Error al procesar un tweet: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//div[@lang]"}
  (Session info: chrome=135.0.7049.96); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x003F8073+60707]
	GetHandleVerifier [0x003F80B4+60772]
	(No symbol) [0x00220683]
	(No symbol) [0x00268660]
	(No symbol) [0x002689FB]
	(No symbol) [0x0025DD81]
	(No symbol) [0x0028D094]
	(No symbol) [0x0025DCA4]
	(No symbol) [0x0028D2C4]
	(No symbol) [0x002AE824]
	(No symbol) [0x0028CE46]
	(No symbol) [0x0025C5D3]
	(No symbol) [0x0025D424]
	GetHandleVerifier [0x0063BB53+2435075]
	GetHandleVerifier [0x006370F3+2416035]
	GetHandleVerifier [0x0065349C+2531660]
	GetHandleVerifier [0x0040F145+155125]
	GetHandleVerifier [0x00415AED+182173]
	GetHandleVerifier [0x003FF948+91640]
	GetHandleVerifier [0x003FFAF0+92064]
	GetHandleVerifier [0x003EA5B0+4704]
	Bas

--------------

De nuevo, utilizamos Selenium para automatizar la búsqueda de tweets en Twitter (X) relacionados con una lista de hashtags específicos sobre el trap argentino. El script configura un navegador Chrome mediante `undetected_chromedriver` para evitar restricciones, accede a la página de búsqueda de Twitter para cada hashtag, realiza desplazamientos hacia abajo para cargar más tweets y extrae información relevante de cada tweet, como el usuario, contenido, idioma, fecha, enlaces, menciones, retweets y likes. Solo se conservan los tweets en español o inglés. Finalmente, los datos recolectados se almacenan en un DataFrame de pandas y se guardan en un archivo CSV para su análisis posterior.

-----------------

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import undetected_chromedriver as uc
import re
from urllib.parse import quote

# Configuración de opciones para Chrome
options = uc.ChromeOptions()
options.add_argument(r"--user-data-dir=C:\Users\adria\AppData\Local\Google\Chrome\User Data")
options.add_argument(r"--profile-directory=Default")

# Iniciar navegador
driver = uc.Chrome(options=options, version_main=135)

keywords_list = ["#TrapArgentino",
    "#BZRP",
    "#Bizarrap",
    "#Duki",
    "#YSYSMODOTRAP",
    "#ModoDiablo",
    "#FreestyleArgentino",
    "#NickiNicole",
    "#Trueno",
    "#TrapArgentino concierto",
     ' #BésameRemix']




tweets_data = []
scrolls = 10

for keyword in keywords_list:
    print(f"🔍 Buscando tweets para: {keyword}")
    search_url = f"https://twitter.com/search?q={quote(keyword)}&src=typed_query&f=live"
    driver.get(search_url)
    time.sleep(10)

    for _ in range(scrolls):
        tweet_elements = driver.find_elements(By.XPATH, '//article[@data-testid="tweet"]')

        for tweet in tweet_elements:
            try:
                # Extraer contenido del tweet con idioma
                content_div = tweet.find_element(By.XPATH, './/div[@lang]')
                lang = content_div.get_attribute('lang')
                
                # Solo conservar tweets en español o inglés
                if lang not in ['es', 'en']:
                    continue

                user = tweet.find_element(By.XPATH, './/span[contains(text(), "@")]').text
                content = content_div.text
                date_time = tweet.find_element(By.XPATH, './/time').get_attribute('datetime')
                links = [link.get_attribute('href') for link in tweet.find_elements(By.XPATH, './/a')]
                mentions = re.findall(r'@\w+', content)
                retweets = tweet.find_element(By.XPATH, './/div[@data-testid="retweet"]').text if tweet.find_elements(By.XPATH, './/div[@data-testid="retweet"]') else '0'
                likes = tweet.find_element(By.XPATH, './/div[@data-testid="like"]').text if tweet.find_elements(By.XPATH, './/div[@data-testid="like"]') else '0'

                if keyword.lower() in content.lower():
                    tweets_data.append({
                        "Palabra Clave": keyword,
                        "Usuario": user,
                        "Contenido": content,
                        "Idioma": lang,
                        "Fecha y Hora": date_time,
                        "Enlaces": links,
                        "Respuestas/Menciones": mentions,
                        "Retweets": retweets,
                        "Likes": likes
                    })

            except Exception as e:
                print(f"⚠️ Error al procesar un tweet: {e}")
                continue

        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(5)

# Guardar los datos en un DataFrame
df = pd.DataFrame(tweets_data)
df.to_csv("tweets_hashtags_trap_argentino.csv", index=False)

print(f"\n✅ Se han recolectado {len(df)} tweets en español o inglés.")
driver.quit()

🔍 Buscando tweets para: #TrapArgentino
🔍 Buscando tweets para: #BZRP
🔍 Buscando tweets para: #Bizarrap
🔍 Buscando tweets para: #Duki
🔍 Buscando tweets para: #YSYSMODOTRAP
🔍 Buscando tweets para: #ModoDiablo
🔍 Buscando tweets para: #FreestyleArgentino
🔍 Buscando tweets para: #NickiNicole
🔍 Buscando tweets para: #Trueno
🔍 Buscando tweets para: #TrapArgentino concierto
🔍 Buscando tweets para:  #BésameRemix

✅ Se han recolectado 240 tweets en español o inglés.


---------------------

A continuación, combinamos los datos de dos archivos CSV, `tweets_trap_argentino.csv` y `tweets_hashtags_trap_argentino.csv`, que contienen información recopilada de tweets relacionados con el trap argentino. Primero, se cargan ambos archivos en dataframes de pandas (`aa` y `bb`). Luego, se concatenan estos dataframes en uno solo, ignorando los índices originales, y se guarda el resultado en un nuevo archivo llamado `tweets_argentinos_combis.csv`.


----------------------

In [ ]:
aa = pd.read_csv('tweets_trap_argentino.csv')
bb = pd.read_csv('tweets_hashtags_trap_argentino.csv')

pd.concat([aa,bb], ignore_index=True).to_csv('tweets_argentinos_combis.csv')

df = pd.read_csv('tweets_argentinos_combis.csv')

In [11]:
df

,Unnamed: 0,Palabra Clave,Usuario,Contenido,Fecha y Hora,Enlaces,Respuestas/Menciones,Retweets,Likes,Idioma
0,0,trap argentino,@FanFutMexa,"se nota que escuchas Trap argentino, que mamas",2025-04-22T06:30:33.000Z,"['https://x.com/FanFutMexa', 'https://x.com/Fa...",[],0,0,NaN
1,1,trap argentino,@Orsosang,"el argentino es lo mas latino que hay, le enca...",2025-04-22T05:00:00.000Z,"['https://x.com/Orsosang', 'https://x.com/Orso...",[],0,0,NaN
2,2,trap argentino,@paprika_trap,"Libertario promedio: ""ño bizito adgentina"" \nE...",2025-04-22T03:43:39.000Z,"['https://x.com/paprika_trap', 'https://x.com/...",[],0,0,NaN
3,3,trap argentino,@ppartedemi_,trap argentino?,2025-04-21T23:28:17.000Z,"['https://x.com/ppartedemi_', 'https://x.com/p...",[],0,0,NaN
4,4,trap argentino,@medici_tiagop,el futuro del trap argentino es ramma ni cabid...,2025-04-21T23:23:50.000Z,"['https://x.com/medici_tiagop', 'https://x.com...",[],0,0,NaN
...,...,...,...,...,...,...,...,...,...,...
1099,1099,#NickiNicole,@rollingstoneEE,"#Miranda con #NickiNicole, #Béele con #MarcAnt...",2025-04-21T23:01:33.000Z,"['https://x.com/rollingstoneEE', 'https://x.co...",[],0,0,es
1100,1100,#NickiNicole,@isabelracal,#HaceRato #Miranda! ft. #NickiNicole https://y...,2025-04-22T03:53:37.000Z,"['https://x.com/isabelracal', 'https://x.com/i...",['@YouTube'],0,0,es
1101,1101,#NickiNicole,@dante_moliterni,"Haciendo estilo realista por primera vez, dibu...",2025-04-22T01:34:53.000Z,"['https://x.com/dante_moliterni', 'https://x.c...",['@Nicki_Nicole19'],0,0,es
1102,1102,#NickiNicole,@rollingstoneEE,"#Miranda con #NickiNicole, #Béele con #MarcAnt...",2025-04-22T01:31:14.000Z,"['https://x.com/rollingstoneEE', 'https://x.co...",[],0,0,es


--------------------

Lo mismo que los anteriores solo que basadas en una lista de palabras clave relacionadas con el trap argentino y sus polémicas.

----------------

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys  # Importar Keys para simular teclas como END
import undetected_chromedriver as uc  # Usar undetected_chromedriver para evitar bloqueos de detección
import re

# Configuración de opciones para el navegador Chrome
options = uc.ChromeOptions()
options.add_argument(r"--user-data-dir=C:\Users\adria\AppData\Local\Google\Chrome\User Data")  # Ruta al perfil de usuario de Chrome
options.add_argument(r"--profile-directory=Default")  # Directorio del perfil predeterminado

# Iniciar el navegador con las opciones configuradas
driver = uc.Chrome(options=options, version_main=135)

# Lista de palabras clave para buscar en Twitter (X)
keywords_list = [
    "Paulo Londra Big Ligas",
    "Paulo Londra demanda",
    "Paulo Londra Warner",
    "Paulo Londra regreso",
    "Paulo Londra juicio",
    "Paulo Londra polémica",
    "Paulo Londra trap",

    "YSYSMO",
    "YSYSMO es un movimiento",
    "YSYSMO no es moda",
    "YSYSMO vs Modo Diablo",
    
    "Modo Diablo separación",
    "Homer El Mero Mero YSY A pelea",
    "Ysy A CRO bardo",
    "Bizarrap y Duki distancia",
    
    "Cazzu Nicki Nicole comparación",
    "Nicki Nicole peso",
    "Nicki Nicole",
    "Nicki Nicole infiel",
    
    "bardo trap argentino",
    "fandoms enfrentados trap argentino",
    "trap argentino toxicidad",
    "hate a artistas trap argetino",
    "twitter trap argentino bardos",
    "trap argentino beef",
    "beef Duki",
    "beef Paulo Londra",
    "beef Bizarrap",
    
    "cancelado trap argentino",
    "cancelación en el trap argentino",
    "cancelar artistas argentinos",
    "trap argentino polémicas 2024",
    
    "tiradera trap argentino",
    "tiradera freestyle argentino",
]

# Lista para almacenar los datos de los tweets recolectados
tweets_data = []

# Número de desplazamientos hacia abajo para cargar más tweets
scrolls = 15

# Bucle para realizar una búsqueda en Twitter para cada palabra clave
for keyword in keywords_list:
    # Construir la URL de búsqueda en Twitter con la palabra clave
    driver.get(f"https://twitter.com/search?q={keyword}&src=typed_query&f=live")
    time.sleep(5)  # Esperar a que la página cargue completamente

    for _ in range(scrolls):
        # Extraer los elementos de los tweets en la página
        tweet_elements = driver.find_elements(By.XPATH, '//article[@data-testid="tweet"]')

        for tweet in tweet_elements:
            try:
                # Extraer el nombre de usuario del tweet
                user = tweet.find_element(By.XPATH, './/span[contains(text(), "@")]').text
                # Extraer el contenido del tweet
                content = tweet.find_element(By.XPATH, './/div[@lang]').text
                # Extraer la fecha y hora del tweet
                date_time = tweet.find_element(By.XPATH, './/time').get_attribute('datetime')
                # Extraer los enlaces presentes en el tweet
                links = [link.get_attribute('href') for link in tweet.find_elements(By.XPATH, './/a')]
                # Extraer las menciones o respuestas en el contenido del tweet
                mentions = re.findall(r'@\w+', content)
                # Extraer el número de retweets (si está disponible)
                retweets = tweet.find_element(By.XPATH, './/div[@data-testid="retweet"]').text if tweet.find_elements(By.XPATH, './/div[@data-testid="retweet"]') else '0'
                # Extraer el número de likes (si está disponible)
                likes = tweet.find_element(By.XPATH, './/div[@data-testid="like"]').text if tweet.find_elements(By.XPATH, './/div[@data-testid="like"]') else '0'
                
                # Guardar los datos extraídos en la lista
                tweets_data.append({
                    "Palabra Clave": keyword,  # Añadir la palabra clave asociada al tweet
                    "Usuario": user,
                    "Contenido": content,
                    "Fecha y Hora": date_time,
                    "Enlaces": links,
                    "Respuestas/Menciones": mentions,
                    "Retweets": retweets,
                    "Likes": likes
                })
            except Exception as e:
                # Manejar errores al procesar un tweet
                print(f"Error al procesar un tweet: {e}")
                continue

        # Simular desplazamiento hacia abajo para cargar más tweets
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(5)

# Crear un DataFrame de pandas con los datos recolectados
df = pd.DataFrame(tweets_data)

# Guardar los datos en un archivo CSV para análisis posterior
df.to_csv("tweets_trap_argentino2.csv", index=False)

# Imprimir el número total de tweets recolectados
print(f"Se han recolectado {len(df)} tweets.")

# Cerrar el navegador
driver.quit()


---------------------

Otra lista de palabras clave relacionadas con el trap argentino y sus polémicas.

---------------------

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import undetected_chromedriver as uc
import re
from urllib.parse import quote

# Configuración de opciones para Chrome
options = uc.ChromeOptions()
options.add_argument(r"--user-data-dir=C:\Users\adria\AppData\Local\Google\Chrome\User Data")  # Ruta al perfil de usuario de Chrome
options.add_argument(r"--profile-directory=Default")  # Directorio del perfil predeterminado

# Iniciar navegador con las opciones configuradas
driver = uc.Chrome(options=options, version_main=135)

# Lista de hashtags para buscar en Twitter
keywords_list = [
    "#YSYSMO",
    "#PauloLondra",
    "#YSYAMovimiento",
    "#BeefTrapArgentino",
    "#ModoDiabloSeAcabó",
    "#FandomsTrapArgentino",
    "#HatersTrapArgentino",
    "#CanceladosTrapArgentino",
]

# Lista para almacenar los datos de los tweets recolectados
tweets_data = []

# Número de desplazamientos hacia abajo para cargar más tweets
scrolls = 20

# Bucle para realizar una búsqueda en Twitter para cada hashtag
for keyword in keywords_list:
    print(f"🔍 Buscando tweets para: {keyword}")
    # Construir la URL de búsqueda en Twitter con el hashtag
    search_url = f"https://twitter.com/search?q={quote(keyword)}&src=typed_query&f=live"
    driver.get(search_url)
    time.sleep(10)  # Esperar a que la página cargue completamente

    # Realizar desplazamientos hacia abajo para cargar más tweets
    for _ in range(scrolls):
        # Extraer los elementos de los tweets en la página
        tweet_elements = driver.find_elements(By.XPATH, '//article[@data-testid="tweet"]')

        for tweet in tweet_elements:
            try:
                # Extraer el contenido del tweet y su idioma
                content_div = tweet.find_element(By.XPATH, './/div[@lang]')
                lang = content_div.get_attribute('lang')
                
                # Solo conservar tweets en español o inglés
                if lang not in ['es', 'en']:
                    continue

                # Extraer el nombre de usuario del tweet
                user = tweet.find_element(By.XPATH, './/span[contains(text(), "@")]').text
                # Extraer el contenido del tweet
                content = content_div.text
                # Extraer la fecha y hora del tweet
                date_time = tweet.find_element(By.XPATH, './/time').get_attribute('datetime')
                # Extraer los enlaces presentes en el tweet
                links = [link.get_attribute('href') for link in tweet.find_elements(By.XPATH, './/a')]
                # Extraer las menciones o respuestas en el contenido del tweet
                mentions = re.findall(r'@\w+', content)
                # Extraer el número de retweets (si está disponible)
                retweets = tweet.find_element(By.XPATH, './/div[@data-testid="retweet"]').text if tweet.find_elements(By.XPATH, './/div[@data-testid="retweet"]') else '0'
                # Extraer el número de likes (si está disponible)
                likes = tweet.find_element(By.XPATH, './/div[@data-testid="like"]').text if tweet.find_elements(By.XPATH, './/div[@data-testid="like"]') else '0'

                # Verificar si el hashtag está presente en el contenido del tweet
                if keyword.lower() in content.lower():
                    # Guardar los datos extraídos en la lista
                    tweets_data.append({
                        "Palabra Clave": keyword,
                        "Usuario": user,
                        "Contenido": content,
                        "Idioma": lang,
                        "Fecha y Hora": date_time,
                        "Enlaces": links,
                        "Respuestas/Menciones": mentions,
                        "Retweets": retweets,
                        "Likes": likes
                    })

            except Exception as e:
                # Manejar errores al procesar un tweet
                print(f"⚠️ Error al procesar un tweet: {e}")
                continue

        # Simular desplazamiento hacia abajo para cargar más tweets
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(5)

# Crear un DataFrame de pandas con los datos recolectados
df = pd.DataFrame(tweets_data)

# Guardar los datos en un archivo CSV para análisis posterior
df.to_csv("tweets_hashtags_trap_argentino2.csv", index=False)

# Imprimir el número total de tweets recolectados
print(f"\n✅ Se han recolectado {len(df)} tweets en español o inglés.")

# Cerrar el navegador
driver.quit()

🔍 Buscando tweets para: #YSYSMO
🔍 Buscando tweets para: #PauloLondra
🔍 Buscando tweets para: #YSYAMovimiento
🔍 Buscando tweets para: #BeefTrapArgentino
🔍 Buscando tweets para: #ModoDiabloSeAcabó
🔍 Buscando tweets para: #FandomsTrapArgentino
🔍 Buscando tweets para: #HatersTrapArgentino
🔍 Buscando tweets para: #CanceladosTrapArgentino

✅ Se han recolectado 200 tweets en español o inglés.


In [ ]:
new = pd.read_csv('tweets_trap_argentino2.csv')
new2 = pd.read_csv('tweets_hashtags_trap_argentino2.csv')
old = pd.read_csv('tweets_argentinos_combis.csv')
pd.concat([new,new2, old], ignore_index=True).to_csv('tweets_argentinos_combis2.csv')